In [1]:
from __future__ import print_function
from __future__ import division

%matplotlib inline

import pandas as pd
import numpy as np
import xgboost as xgb

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# just for the sake of this blog post!
from warnings import filterwarnings
filterwarnings('ignore')


In [2]:
# load the provided data
train_features = pd.read_csv('data-processed/dengue_features_train.csv',
                             index_col=[0,1,2])

train_labels = pd.read_csv('data-processed/dengue_labels_train.csv',
                           index_col=[0,1,2])


In [3]:
# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']



In [4]:
print('San Juan')
print('features: ', sj_train_features.shape)
print('labels  : ', sj_train_labels.shape)

print('\nIquitos')
print('features: ', iq_train_features.shape)
print('labels  : ', iq_train_labels.shape)

San Juan
features:  (936, 22)
labels  :  (936, 1)

Iquitos
features:  (520, 22)
labels  :  (520, 1)


In [5]:
# Remove `week_start_date` string.
sj_train_features.drop('week_start_date', axis=1, inplace=True)
iq_train_features.drop('week_start_date', axis=1, inplace=True)



In [6]:
print('San Juan')
print('mean: ', sj_train_labels.mean()[0])
print('var :', sj_train_labels.var()[0])

print('\nIquitos')
print('mean: ', iq_train_labels.mean()[0])
print('var :', iq_train_labels.var()[0])

San Juan
mean:  34.18055555555556
var : 2640.0454396910277

Iquitos
mean:  7.565384615384615
var : 115.89552393656439


In [7]:
sj_train_features.isnull().any(axis=0)

ndvi_ne                                  False
ndvi_nw                                  False
ndvi_se                                  False
ndvi_sw                                  False
precipitation_amt_mm                     False
reanalysis_air_temp_k                    False
reanalysis_avg_temp_k                    False
reanalysis_dew_point_temp_k              False
reanalysis_max_air_temp_k                False
reanalysis_min_air_temp_k                False
reanalysis_precip_amt_kg_per_m2          False
reanalysis_relative_humidity_percent     False
reanalysis_sat_precip_amt_mm             False
reanalysis_specific_humidity_g_per_kg    False
reanalysis_tdtr_k                        False
station_avg_temp_c                       False
station_diur_temp_rng_c                  False
station_max_temp_c                       False
station_min_temp_c                       False
station_precip_mm                        False
months                                   False
dtype: bool

In [8]:
iq_train_features.isnull().any(axis=0)

ndvi_ne                                  False
ndvi_nw                                  False
ndvi_se                                  False
ndvi_sw                                  False
precipitation_amt_mm                     False
reanalysis_air_temp_k                    False
reanalysis_avg_temp_k                    False
reanalysis_dew_point_temp_k              False
reanalysis_max_air_temp_k                False
reanalysis_min_air_temp_k                False
reanalysis_precip_amt_kg_per_m2          False
reanalysis_relative_humidity_percent     False
reanalysis_sat_precip_amt_mm             False
reanalysis_specific_humidity_g_per_kg    False
reanalysis_tdtr_k                        False
station_avg_temp_c                       False
station_diur_temp_rng_c                  False
station_max_temp_c                       False
station_min_temp_c                       False
station_precip_mm                        False
months                                   False
dtype: bool

In [9]:
sj_train_features['total_cases'] = sj_train_labels.total_cases
iq_train_features['total_cases'] = iq_train_labels.total_cases

In [10]:
sj_train_features.head()

ndvi_ne   ndvi_nw   ndvi_se   ndvi_sw  precipitation_amt_mm  \
year weekofyear                                                                 
1990 18          0.122600  0.103725  0.198483  0.177617                 12.42   
     19          0.169900  0.142175  0.162357  0.155486                 22.82   
     20          0.032250  0.172967  0.157200  0.170843                 34.54   
     21          0.128633  0.245067  0.227557  0.235886                 15.36   
     22          0.196200  0.262200  0.251200  0.247340                  7.52   

                 reanalysis_air_temp_k  reanalysis_avg_temp_k  \
year weekofyear                                                 
1990 18                     297.572857             297.742857   
     19                     298.211429             298.442857   
     20                     298.781429             298.878571   
     21                     298.987143             299.228571   
     22                     299.518571             299.664286   

                 reanalysis_dew_point_temp_k  reanalysis_max_air_temp_k  \
year weekofyear                                                           
1990 18                           292.414286                      299.8   
     19                           293.951429                      300.9   
     20                           295.434286                      300.5   
     21                           295.310000                      301.4   
     22                           295.821429                      301.9   

                 reanalysis_min_air_temp_k  ...  reanalysis_sat_precip_amt_mm  \
year weekofyear                             ...                                 
1990 18                              295.9  ...                         12.42   
     19                              296.4  ...                         22.82   
     20                              297.3  ...                         34.54   
     21                              297.0  ...                         15.36   
     22                              297.5  ...                          7.52   

                 reanalysis_specific_humidity_g_per_kg  reanalysis_tdtr_k  \
year weekofyear                                                             
1990 18                                      14.012857           2.628571   
     19                                      15.372857           2.371429   
     20                                      16.848571           2.300000   
     21                                      16.672857           2.428571   
     22                                      17.210000           3.014286   

                 station_avg_temp_c  station_diur_temp_rng_c  \
year weekofyear                                                
1990 18                   25.442857                 6.900000   
     19                   26.714286                 6.371429   
     20                   26.714286                 6.485714   
     21                   27.471429                 6.771429   
     22                   28.942857                 9.371429   

                 station_max_temp_c  station_min_temp_c  station_precip_mm  \
year weekofyear                                                              
1990 18                        29.4                20.0               16.0   
     19                        31.7                22.2                8.6   
     20                        32.2                22.8               41.4   
     21                        33.3                23.3                4.0   
     22                        35.0                23.9                5.8   

                 months  total_cases  
year weekofyear                       
1990 18               4            4  
     19               5            5  
     20               5            4  
     21               5            3  
     22               5            6  

[5 rows x 22 columns]

In [11]:
sj_train_features.to_csv("data-processed/sj_train_features.csv")
iq_train_features.to_csv("data-processed/iq_train_features.csv")

In [12]:
sj_train_features.isnull().sum(axis = 0)

ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
months                                   0
total_cases                              0
dtype: int64

In [13]:
iq_train_features.isnull().sum(axis = 0)

ndvi_ne                                  0
ndvi_nw                                  0
ndvi_se                                  0
ndvi_sw                                  0
precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
months                                   0
total_cases                              0
dtype: int64

In [14]:
sj_train_features = sj_train_features.drop(columns = ['ndvi_ne'])
sj_train_features = sj_train_features.drop(columns = ['ndvi_nw'])
sj_train_features = sj_train_features.drop(columns = ['ndvi_se'])
sj_train_features = sj_train_features.drop(columns = ['ndvi_sw'])
sj_train_features.isnull().sum(axis = 0)

precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
station_precip_mm                        0
months                                   0
total_cases                              0
dtype: int64

In [15]:
iq_train_features = sj_train_features.drop(columns = ['station_avg_temp_c'])
iq_train_features = sj_train_features.drop(columns = ['station_diur_temp_rng_c'])
iq_train_features = sj_train_features.drop(columns = ['station_max_temp_c'])
iq_train_features = sj_train_features.drop(columns = ['station_precip_mm'])
iq_train_features.isnull().sum(axis = 0)

precipitation_amt_mm                     0
reanalysis_air_temp_k                    0
reanalysis_avg_temp_k                    0
reanalysis_dew_point_temp_k              0
reanalysis_max_air_temp_k                0
reanalysis_min_air_temp_k                0
reanalysis_precip_amt_kg_per_m2          0
reanalysis_relative_humidity_percent     0
reanalysis_sat_precip_amt_mm             0
reanalysis_specific_humidity_g_per_kg    0
reanalysis_tdtr_k                        0
station_avg_temp_c                       0
station_diur_temp_rng_c                  0
station_max_temp_c                       0
station_min_temp_c                       0
months                                   0
total_cases                              0
dtype: int64

In [16]:
sj_train_features.to_csv("data-processed/sj_train_features_droped.csv")
iq_train_features.to_csv("data-processed/iq_train_features_droped.csv")

In [17]:
def preprocess_data(data_path, labels_path=None):
    # load data and set index to city, year, weekofyear
    df = pd.read_csv(data_path, index_col=[0, 1, 2])
    
    # select features we want
    features=['precipitation_amt_mm',
                     'reanalysis_air_temp_k',
                     'reanalysis_avg_temp_k',
                     'reanalysis_max_air_temp_k',
                     'reanalysis_relative_humidity_percent',
                     'station_avg_temp_c',
                     'station_max_temp_c']
    df = df[features]
    
    # fill missing values
    df.fillna(df.mean(), inplace=True)

    # add labels to dataframe
    if labels_path:
        labels = pd.read_csv(labels_path, index_col=[0, 1, 2])
    
    # separate san juan and iquitos
    sj_train = df.loc['sj']
    iq_train = df.loc['iq']
    
    if(labels_path!=None):
        sj_labels = labels.loc['sj']
        iq_labels = labels.loc['iq']
    
        return sj_train, iq_train,sj_labels,iq_labels
    else:
        return sj_train, iq_train

In [18]:
sj_train, iq_train,sj_labels,iq_labels= preprocess_data('data-processed/dengue_features_train.csv',labels_path="data-processed/dengue_labels_train.csv")

In [19]:
X_train_sj, X_test_sj, y_train_sj, y_test_sj = train_test_split(sj_train, sj_labels, test_size=0.2, random_state=123)

In [20]:
#sj_labels = np.array(sj_labels)
sj_labels = sj_labels

In [21]:
#sj_features = np.array(sj_train)
sj_features = sj_train

In [22]:
train_features_sj, test_features_sj, train_labels_sj, test_labels_sj = train_test_split(sj_features, sj_labels, test_size = 0.2, random_state = 42)
train_features_iq, test_features_iq, train_labels_iq, test_labels_iq = train_test_split(iq_train, iq_labels, test_size = 0.2, random_state = 42)

In [23]:
print('Training Features Shape:', train_features_sj.shape)
print('Training Labels Shape:', train_labels_sj.shape)
print('Testing Features Shape:', test_features_sj.shape)
print('Testing Labels Shape:', test_labels_sj.shape)

Training Features Shape: (748, 7)
Training Labels Shape: (748, 1)
Testing Features Shape: (188, 7)
Testing Labels Shape: (188, 1)


In [24]:
# Instantiate model with 1000 decision trees
rf_sj = RandomForestRegressor(n_estimators=1000, max_depth=10,random_state=140)
# Train the model on training data
rf_sj.fit(sj_features, sj_labels);

In [25]:
sj_test, iq_test = preprocess_data('data-processed/dengue_features_test.csv')


In [26]:
predictions_random_forest_sj = rf_sj.predict(sj_test).astype(int)
predictions_random_forest_sj_test = rf_sj.predict(test_features_sj).astype(int)

In [27]:
regressor_sj = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.1, max_depth = 10, alpha = 10, n_estimators = 100)
regressor_sj.fit(sj_features, sj_labels)

predictions_XGBoost_sj = regressor_sj.predict(sj_test).astype(int)
predictions_XGBoost_sj_test = regressor_sj.predict(test_features_sj).astype(int)

In [28]:
print("Make predictions on the test set")
test_probs_sj = ((predictions_random_forest_sj + predictions_XGBoost_sj)/2).astype(int)
test_probs_sj_test = ((predictions_random_forest_sj_test + predictions_XGBoost_sj_test)/2).astype(int)

Make predictions on the test set


In [29]:
# Instantiate model with 1000 decision trees
rf_sj = RandomForestRegressor(n_estimators=1000, max_depth=10,random_state=140)
# Train the model on training data
rf_sj.fit(iq_train, iq_labels);

In [30]:
predictions_random_forest_iq = rf_sj.predict(iq_test).astype(int)
predictions_random_forest_iq_test = rf_sj.predict(test_features_iq).astype(int)

In [31]:
regressor_iq = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 1, learning_rate = 0.1, max_depth = 10, alpha = 10, n_estimators = 1000)
regressor_iq.fit(iq_train, iq_labels)

predictions_XGBoost_iq = regressor_iq.predict(iq_test).astype(int)
predictions_XGBoost_iq_test = regressor_iq.predict(test_features_iq).astype(int)

In [32]:
print("Make predictions on the test set")
test_probs_iq = ((predictions_random_forest_iq + predictions_XGBoost_iq)/2).astype(int)
test_probs_iq_test = ((predictions_random_forest_iq_test + predictions_XGBoost_iq_test)/2).astype(int)


Make predictions on the test set


In [33]:
MAE_sj = mean_absolute_error(test_probs_sj_test, test_labels_sj )
print("MAE_sj: %f" % (MAE_sj))

MAE_sj: 8.659574


In [34]:
MAE_iq = mean_absolute_error(test_probs_iq_test, test_labels_iq )
print("MAE_iq: %f" % (MAE_iq))

MAE_iq: 1.692308


In [35]:
submission = pd.read_csv("data-processed/submission_format.csv",
                         index_col=[0, 1, 2])

submission.total_cases = np.concatenate([test_probs_sj, test_probs_iq])
submission.to_csv("data-processed/benchmark.csv")